### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
num_schools = school_data["school_name"].count()
student_tot = school_data_complete["student_name"].count()
budg_tot = school_data["budget"].sum()
avg_math = school_data_complete["math_score"].mean()
avg_read = school_data_complete["reading_score"].mean()
#making pass bin fails for passing percentage calculations
pass_bin = [0,70,100]
pass_labels = ["Fail","Pass"]
school_data_complete["read_res"] = pd.cut(school_data_complete["reading_score"], pass_bin, labels=pass_labels)
school_data_complete["math_res"] = pd.cut(school_data_complete["math_score"], pass_bin, labels=pass_labels)
#calculating the passing for each subject
read_pass_count = (school_data_complete["read_res"] == "Pass").sum()
read_pass_perc = (read_pass_count / student_tot)
math_pass_count = (school_data_complete["math_res"] == "Pass").sum()
math_pass_perc = (math_pass_count / student_tot)
combo_pass_count = ((school_data_complete["read_res"] == "Pass") & (school_data_complete["math_res"] == "Pass")).sum()
combo_pass_perc = (combo_pass_count / student_tot)
#formatting
dist_summ_df = pd.DataFrame([[num_schools,student_tot,budg_tot,avg_math,math_pass_perc,avg_read,read_pass_perc,combo_pass_perc]], columns=["Total Schools","Total Students","Total Budget","Average Math Score","% Passing Math","Average Reading Score","% Passing Reading","% Overall Passing"])
form_dic = {"Total Students":"{:,.0f}", "Total Budget":"${0:,.2f}", "Average Math Score":"{:,.2f}", "Average Reading Score":"{:,.2f}", "% Passing Math":"{:.2%}", "% Passing Reading":"{:.2%}", "% Overall Passing":"{:.2%}", "Total School Budget":"${0:,.2f}", "Per Student Budget":"${0:,.2f}", "9th":"{:,.2f}", "10th":"{:,.2f}", "11th":"{:,.2f}", "12th":"{:,.2f}"}


dist_summ_df.style.format(form_dic)

,Total Schools,Total Students,Total Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,72.39%,81.88,82.97%,60.80%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#setting initial data frames to work off of
school_data_complete = school_data_complete.rename(columns={"school_name":"School Name"})
group_schools = school_data_complete.groupby("School Name")
group_schools_2 = school_data_complete.set_index("School Name")
#pulling specific data to use in making columns
    #type and budget columns data
group_sch_set_vars = group_schools_2.loc[ : , ["type", "budget"]]
    #percentage of passing student data
pass_fail_read = group_schools_2.loc[group_schools_2["read_res"] != "Fail"]
pass_fail_math = group_schools_2.loc[group_schools_2["math_res"] != "Fail"]
pass_fail_combo =  group_schools_2.loc[(group_schools_2["math_res"] != "Fail") & (group_schools_2["read_res"] != "Fail")]
read_pass_ss = pass_fail_read.groupby("School Name").agg(read_pass_count = pd.NamedAgg(column="student_name", aggfunc="count"))
math_pass_ss = pass_fail_math.groupby("School Name").agg(math_pass_count = pd.NamedAgg(column="student_name", aggfunc="count"))
combo_pass_ss = pass_fail_combo.groupby("School Name").agg(combo_pass_count = pd.NamedAgg(column="student_name", aggfunc="count"))
    #average of scores data
group_sch_avgs = group_schools[["math_score", "reading_score"]].mean()
    #student count data
group_sch_stu_count = group_schools.agg(Total_Students = pd.NamedAgg(column="student_name", aggfunc="count"))
#merging all data into one data frame
mer_sch_summ = pd.merge(pd.merge(group_sch_avgs,group_sch_stu_count, on="School Name"),group_sch_set_vars, on="School Name", how="left").drop_duplicates()
mer_sch_summ["Per Student Budget"] = mer_sch_summ["budget"]/mer_sch_summ["Total_Students"]
mer_sch_summ["% Passing Math"] = math_pass_ss["math_pass_count"] / group_sch_stu_count["Total_Students"]
mer_sch_summ["% Passing Reading"] = read_pass_ss["read_pass_count"] / group_sch_stu_count["Total_Students"]
mer_sch_summ["% Overall Passing"] = combo_pass_ss["combo_pass_count"] / group_sch_stu_count["Total_Students"]
#Formatting
#clean_Sch_Summ = pd.DataFrame([[group_sch_set_vars[["type"]],group_sch_stu_count[["Total_Students"]],group_sch_set_vars[["budget"]],mer_sch_summ[["Per Student Budget"]],group_sch_avgs[["math_score", "reading_score"]],mer_sch_summ[["% Passing Math","% Passing Reading","% Overall Passing"]]]], columns=["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"])
mer_sch_summ = mer_sch_summ.reindex(columns=["type", "Total_Students", "budget", "Per Student Budget", "math_score", "reading_score", "% Passing Math", "% Passing Reading", "% Overall Passing"])
mer_sch_summ_form = mer_sch_summ.rename(columns={"type":"School Type","Total_Students":"Total Students","budget":"Total School Budget","math_score":"Average Math Score","reading_score":"Average Reading Score"})

mer_sch_summ_form.style.format(form_dic)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,64.63%,79.30%,51.15%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,89.56%,93.86%,84.02%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,63.75%,78.43%,49.92%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,65.75%,77.51%,51.30%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,89.71%,93.39%,83.65%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,64.75%,78.19%,50.16%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,90.63%,92.74%,84.07%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,63.32%,78.81%,49.91%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,63.85%,78.28%,49.80%


In [4]:
## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
mer_sch_summ_form.sort_values("% Overall Passing",ascending=False).head().style.format(form_dic)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.27,83.99,90.93%,93.25%,84.89%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,91.68%,92.20%,84.82%
Wright High School,Charter,"1,800","$1,049,400.00",$583.00,83.68,83.95,90.28%,93.44%,84.44%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.42,83.85,90.21%,92.91%,84.28%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,90.63%,92.74%,84.07%


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
mer_sch_summ_form.sort_values("% Overall Passing").head().style.format(form_dic)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,64.07%,77.74%,49.44%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,63.85%,78.28%,49.80%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,63.32%,78.81%,49.91%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,63.75%,78.43%,49.92%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,64.75%,78.19%,50.16%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
ninth = group_schools_2.query("grade=='9th'")
ninth = ninth.filter(["School Name", "math_score"])
math_sco_grade = ninth.groupby(["School Name"]).mean()
math_sco_grade = math_sco_grade.rename(columns={"math_score":"9th"})
tenth = group_schools_2.query("grade=='10th'")
tenth = tenth.filter(["School Name", "math_score"])
math_sco_grade["10th"] = tenth.groupby(["School Name"]).mean()
eleventh = group_schools_2.query("grade=='11th'")
eleventh = eleventh.filter(["School Name", "math_score"])
math_sco_grade["11th"] = eleventh.groupby(["School Name"]).mean()
twelth = group_schools_2.query("grade=='12th'")
twelth = twelth.filter(["School Name", "math_score"])
math_sco_grade["12th"] = twelth.groupby(["School Name"]).mean()


#school_data_complete.head()
math_sco_grade.style.format(form_dic)

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
ninth = group_schools_2.query("grade=='9th'")
ninth = ninth.filter(["School Name", "reading_score"])
read_sco_grade = ninth.groupby(["School Name"]).mean()
read_sco_grade = read_sco_grade.rename(columns={"reading_score":"9th"})
tenth = group_schools_2.query("grade=='10th'")
tenth = tenth.filter(["School Name", "reading_score"])
read_sco_grade["10th"] = tenth.groupby(["School Name"]).mean()
eleventh = group_schools_2.query("grade=='11th'")
eleventh = eleventh.filter(["School Name", "reading_score"])
read_sco_grade["11th"] = eleventh.groupby(["School Name"]).mean()
twelth = group_schools_2.query("grade=='12th'")
twelth = twelth.filter(["School Name", "reading_score"])
read_sco_grade["12th"] = twelth.groupby(["School Name"]).mean()

read_sco_grade.style.format(form_dic)

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
spend_bin = [0,585,630,645,680]
spend_labels = ["<$585","$585-630","$630-645","$645-680"]
mer_sch_summ_form["Spending Ranges (Per Student)"] = pd.cut(mer_sch_summ["Per Student Budget"], spend_bin, labels=spend_labels)
school_spend = mer_sch_summ_form.filter(["Spending Ranges (Per Student)", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"])
school_spend = school_spend.groupby(["Spending Ranges (Per Student)"]).mean()
school_spend.style.format(form_dic)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,90.35%,93.33%,84.36%
$585-630,81.90,83.16,83.98%,89.38%,75.70%
$630-645,78.52,81.62,70.95%,81.65%,58.73%
$645-680,77.00,81.03,63.97%,78.43%,49.96%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
size_bin = [0,1000,2000,5000]
size_labels = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]
mer_sch_summ_form["School Size"] = pd.cut(mer_sch_summ["Total_Students"], size_bin, labels=size_labels)
school_size = mer_sch_summ_form.filter(["School Size", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"])
school_size = school_size.groupby(["School Size"]).mean()
school_size.style.format(form_dic)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.82,83.93,91.16%,92.47%,84.45%
Medium(1000-2000),83.37,83.86,89.93%,93.24%,83.92%
Large(2000-5000),77.75,81.34,67.63%,80.19%,54.57%


## Scores by School Type

* Perform the same operations as above, based on school type

In [11]:
#type_bin = [0,70,100]
#type_labels = ["Fail","Pass"]

school_type = pd.pivot_table(mer_sch_summ_form, values=["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"], index="School Type", aggfunc=np.mean, sort=False )
school_type = school_type.reindex(columns=["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"])
school_type.style.format(form_dic)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
District,76.96,80.97,64.30%,78.32%,50.24%
Charter,83.47,83.90,90.36%,93.05%,84.17%
